In [1]:
import jieba
import re
import pymongo
from gensim import corpora, models, similarities
jieba.set_dictionary("dict.txt.big.txt")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [2]:
ettoday_list = []
for i in mycol.find({},{"title":1,"_id":0}):
    ettoday_list.append(i["title"])
print(ettoday_list[:10])

['快訊／高嘉瑜晚間好奇找阿北求證\u3000柯P終於「一句話」解答了', '美眉的夏天／最HOT名單出爐\u3000她激戰搶下冠軍抱千元大獎', '搭西寧軍艦南偵\u300022名教師登太平島深化主權意識', '桃園區慶讚中元水燈排車繞境\u3000傳承百年特色慶典', '鳳飛飛故事展、電影展…等紀念活動\u3000緬懷永遠的國民天后', '韓國瑜選總統8月3日從桃園出發\u3000要讓全國看到藍軍團結力量', '規劃「個人服裝包」\u3000國軍新兵服裝用APP快速搞定', '嘉義「火燈夜巡、大士爺祭」深獲在地認同\u3000下一步推向國際', '台民黨「不分區名單」她第一個被點名！\u3000柯文哲傾向不選才組黨', '影／她吃藥睡死…起床驚見「手被蟑螂咬」模樣超慘\u3000網加碼曝：而且非常痛']


In [3]:
def clean_data(title):
    sub_t = "\W+"
    t_sub = re.sub(sub_t,"",title)
    t_sub = re.sub("快訊","",t_sub)
    return list(jieba.cut(t_sub))

ettoday_list = [clean_data(t) for t in ettoday_list]
print(ettoday_list[:10])

Building prefix dict from D:\program\semester_final\dict.txt.big.txt ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.u202fd84447beda33a25aefd770cfa382.cache
Loading model cost 1.280 seconds.
Prefix dict has been built succesfully.


[['高嘉瑜', '晚間', '好奇', '找', '阿北', '求證', '柯', 'P', '終於', '一句', '話', '解答', '了'], ['美眉', '的', '夏天', '最', 'HOT', '名單', '出爐', '她', '激戰', '搶下', '冠軍', '抱', '千元', '大獎'], ['搭', '西寧', '軍艦', '南偵', '22', '名', '教師', '登', '太平島', '深化', '主權', '意識'], ['桃園', '區', '慶', '讚', '中', '元', '水燈', '排車', '繞境', '傳承', '百年', '特色', '慶典'], ['鳳飛飛', '故事', '展', '電影展', '等', '紀念活動', '緬懷', '永遠', '的', '國民', '天后'], ['韓國', '瑜', '選', '總統', '8', '月', '3', '日', '從', '桃園', '出發', '要', '讓', '全國', '看到', '藍', '軍', '團結', '力量'], ['規劃', '個人', '服裝', '包', '國軍', '新兵', '服裝', '用', 'APP', '快速', '搞定'], ['嘉義', '火燈', '夜巡', '大士', '爺祭', '深獲', '在', '地', '認同', '下', '一步', '推向', '國際'], ['台', '民黨', '不', '分區', '名單', '她', '第一個', '被', '點名', '柯文', '哲', '傾向', '不選才', '組黨'], ['影', '她', '吃藥', '睡死', '起床', '驚見', '手', '被', '蟑螂', '咬', '模樣', '超', '慘網', '加碼', '曝', '而且', '非常', '痛']]


In [4]:
corpora_dict_path = './model/corpora_dict.dict'
tfidf_model_path = "./model/tfidf.model"
index_path = "./model/simIndex.index"
lsi_model_path = "./model/lsi_model.lsi"

In [5]:
dictionary = corpora.Dictionary(ettoday_list)
dictionary.save(corpora_dict_path)
corpus = [dictionary.doc2bow(text) for text in ettoday_list]

In [6]:
tfidf = models.TfidfModel(corpus)
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=300)
index = similarities.SparseMatrixSimilarity(lsi[corpus],num_features = len(dictionary.token2id))

In [7]:
tfidf.save(tfidf_model_path)
index.save(index_path)
lsi.save(lsi_model_path)

In [4]:
# dictionary = corpora.Dictionary(ettoday_list)
# dictionary.save(corpora_dict_path)

# corpus = [dictionary.doc2bow(text) for text in ettoday_list]
# # corpora.MmCorpus.serialize('./corpus.mm', corpus) 

# tfidf = models.TfidfModel(corpus)
# tfidf.save(tfidf_model_path)

# index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.token2id))
# index.save(index_path)

****

In [1]:
import jieba
import re
import pymongo
from gensim import corpora, models, similarities
import pandas as pd
jieba.set_dictionary("dict.txt.big.txt")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [2]:
from keras.models import load_model
import keras
model = load_model("./model/LSTM_model3.h5")

Using TensorFlow backend.


In [3]:
import pickle
with open("./clean_data/tokenized2.pickle","rb") as handle:
    tokenizer = pickle.load(handle)

In [4]:
corpora_dict_path = './model/corpora_dict.dict'
tfidf_model_path = "./model/tfidf.model"
index_path = "./model/simIndex.index"
lsi_model_path = "./model/lsi_model.lsi"

tfidf = models.TfidfModel.load(tfidf_model_path)
dictionary = corpora.Dictionary.load(corpora_dict_path)
index = similarities.SparseMatrixSimilarity.load(index_path)
lsi =  models.LsiModel.load(lsi_model_path)

In [5]:
def cleanData(sentence):
    sentence = re.sub("快訊","",sentence)
    sentence = re.sub("\W+","",sentence)
    texts = jieba.cut(sentence)
    arr = []
    for t in texts:
        if(t.isdigit() or re.search("[^a-zA-Z]+",t) == None):
            arr.append(t)
            continue
        else:
            for tee in t:
                arr.append(tee)
    return " ".join(arr)

def stringify(text):
    return str(text)

In [6]:
r = '\W+'
def clean_mark(text):
    text = re.sub("快訊","",text)
    text = re.sub(r,"",text)
    words = jieba.cut(text)
    return " ".join([word for word in words])
def stringify(text):
    return str(text)

In [7]:
all_ettoday = []
for i in mycol.find({},{"title":1}):
    all_ettoday.append(i)

In [8]:
def getSims(sentence):
    texts = jieba.cut(sentence)
    new_vec = dictionary.doc2bow(texts)
    new_vec_tfidf = tfidf[new_vec]
    new_vec_lsi = lsi[new_vec_tfidf]
    return index[new_vec_lsi]

In [9]:
user_input = "螺絲起子直接刮MINI百萬愛車！"

In [10]:
def getTFIDF(sentence):
    texts = jieba.cut(sentence)
    new_vec = dictionary.doc2bow(texts)
    new_vec_tfidf = tfidf[new_vec]
    return index[new_vec_tfidf]

In [15]:
sims = getS(user_input)
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [16]:
print(getTFIDF[:10])
[print(all_ettoday[sim[0]]) for sim in sims[:10]]

[(23761, 0.5972205), (35902, 0.5773784), (39645, 0.5623006), (15875, 0.5001459), (16829, 0.46768466), (29716, 0.45888713), (6844, 0.4438838), (26995, 0.4154801), (19936, 0.40443608), (48365, 0.40166098)]
{'_id': ObjectId('5da6f18459aa7d15bc182729'), 'title': '螺絲起子直接刮MINI百萬愛車！\u3000蔡阿嘎一刀未剪實測「＄899防刮神器」：ＸＸ娘咧'}
{'_id': ObjectId('5da6f49f59aa7d15bc185696'), 'title': '鳳鼻隧道限高4.6m…曳引車「繞道強闖」卡住！頂部電纜毀恐賠百萬'}
{'_id': ObjectId('5da6f5a659aa7d15bc186535'), 'title': '保險業App首創！南山人壽「直接串接」健保署資料庫'}
{'_id': ObjectId('5da6ef9059aa7d15bc18085b'), 'title': '楊慶煌百萬名車遭路樹壓垮\u3000窗戶殘骸飛滿地心痛'}
{'_id': ObjectId('5da6efc159aa7d15bc180c15'), 'title': '馬東石新作揪張基龍打爆罪犯！金亞中轉身辣翻\u3000預告點破百萬'}
{'_id': ObjectId('5da6f31659aa7d15bc183e6c'), 'title': '蜂農痛心！\u3000養蜂場314蜂箱500萬隻蜂「遭下毒」損失破百萬'}
{'_id': ObjectId('5da6ed7f59aa7d15bc17e514'), 'title': '基隆大樓突降恐怖「玻璃雨」\u3000百萬名車Mini Cooper衰遭砸'}
{'_id': ObjectId('5da6f25b59aa7d15bc1833cb'), 'title': '正宮心意決...砸百萬曝「超胸照」連載搭機離台！頭版政治淪背景\u3000網：求後續'}
{'_id': ObjectId('5da6f09759aa7d15bc181838'), '

[None, None, None, None, None, None, None, None, None, None]

In [27]:
input_id = [all_ettoday[sim[0]]["_id"] for sim in sims[:5]]
# print(input_id)
for i in input_id:
    print(mycol.find_one({"_id":i}))
# for i in mycol.find({"_id":{"$in":input_id}},{"title":1,"_id":1,"link":1,"img_path":1}):
    

{'_id': ObjectId('5da6f18459aa7d15bc182729'), 'title': '螺絲起子直接刮MINI百萬愛車！\u3000蔡阿嘎一刀未剪實測「＄899防刮神器」：ＸＸ娘咧', 'content': '記者張筱涵／綜合報導\nYouTuber蔡阿嘎推出的新單元「網路購物大破解」來到第三集，這次要實測網路上廣告超多的價值台幣899元防刮神器，看見廣告內容說是「在刮痕上塗一塗就不用去烤漆」，他還特別買威力加強版，接著親自拿螺絲起子刮自己的MINI百萬愛車做實驗，結果讓他大飆髒話。\n蔡阿嘎在網友推薦下花了台幣899元買威力加強版的防刮神器，一開始先塗在本身就被刮到的小刮痕上測試，因為老闆有交代「先用海綿圓圓地推開且要有耐心」，他連續塗抹5分鐘後拿出抹布，強調一刀未剪且要攝影師鏡頭拉近對焦，結果一把泥擦掉，一看到刮痕還在：「一模一樣，ＸＸ娘咧。」\n因為前面有和大頭佛說好要親自刮車測試，蔡阿嘎拿出螺絲起子刮了大約5公分的刮痕，馬上拿防刮神器同樣抹5分鐘，最後還是一點變化都沒有讓他氣得飆罵：「ＸＸ娘咧！真的沒有效！如果老闆你覺得我弄錯了，你他Ｘ的下面留言我看你怎麼弄！」\n同樣有買過的網友也怒分享「對！我他Ｘ買過，塗了快20分鐘，除了車子一堆白膠，什麼Ｘ用都沒有」，同時原本有打算買的網友也跟著感謝蔡阿嘎實測。另外，部分做相關產業的網友也特別留言指出，「我爸做汽車烤漆的常常在說，如果有這麼簡單用掉我們這些烤漆師傅就不用生存了」、「我是一個汽車美容的店家！感謝這部影片拍給一堆腦弱的人看！」', 'time': '2019-09-06_11-07-00', 'tag': '影劇', 'link': 'https://www.ettoday.net/news/20190906/1529767.htm', 'img_path': './pic/20190906/2019-09-06_11-07-00_1529767.png'}
{'_id': ObjectId('5da6f49f59aa7d15bc185696'), 'title': '鳳鼻隧道限高4.6m…曳引車「繞道強闖」卡住！頂部電纜毀恐賠百萬', 'content': '記者陳凱力、莊智勝／新竹報導\n新竹一名52歲的劉姓男子25日凌晨2時許駕駛曳引車行經鳳鼻隧道，明知車身高度過高，但劉男仍是

In [ ]:
top_num = 10
input_data = [all_ettoday[sim[0]]["title"] for sim in sims[:top_num]]
# print(input_data)
input_data = pd.DataFrame(input_data)
input_data.columns = ["title2"]
input_data["title1"] = user_input
input_data.head()

In [ ]:
MAX_SEQUENCE_LENGTH = 25
input_data['title1_tokenized'] = \
    input_data.loc[:, 'title1'].apply(stringify).apply(cleanData)
input_data['title2_tokenized'] = \
    input_data.loc[:, 'title2'].apply(stringify).apply(cleanData)

# 將詞彙序列轉為索引數字的序列
x1_test = tokenizer \
    .texts_to_sequences(
        input_data.title1_tokenized)
x2_test = tokenizer \
    .texts_to_sequences(
        input_data.title2_tokenized)

# 為數字序列加入 zero padding
x1_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(
        x1_test, 
        maxlen=MAX_SEQUENCE_LENGTH)
x2_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(
        x2_test, 
        maxlen=MAX_SEQUENCE_LENGTH)    

# 利用已訓練的模型做預測
predictions = model.predict(
    [x1_test, x2_test])

In [ ]:
import numpy as np
label_to_index = {
    'unrelated': 0, 
    'agreed': 1, 
    'disagreed': 2
}
index_to_label = {v: k for k, v in label_to_index.items()}

input_data['Category'] = [index_to_label[idx] for idx in np.argmax(predictions, axis=1)]
input_data.head(50)

In [ ]:
label_2_num = {
    0: 0, 
    1: 1, 
    2: -1
}
lsi_index = [sim[0] for sim in sims[:top_num]]
lsi_sims = [sim[1] for sim in sims[:top_num]]
pred_label = [label_2_num[idx] for idx in np.argmax(predictions, axis=1)]

score = 0.

for i in range(len(lsi_index)):
    score += lsi_sims[i]*pred_label[i]
    
print(score)

In [ ]:
print(pred_label)
print(lsi_sims)
print(np.argmax(predictions, axis=1))
print(predictions)
print(predictions[range(10),np.argmax(predictions, axis=1)])

In [ ]:
str(round(0.7069477,2))

In [17]:
def judgeScore(score):
    if(score<2.5):
        return "我們猜應該是假新聞"
    elif(score<5 and score >=2.5):
        return "可能新聞不在新聞資料庫內，或是有爭議無法評定"
    else:
        return "我們認為是真新聞"

*****

In [1]:
import jieba
import pymongo
import re
import keras
import numpy as np
from gensim import corpora, models, similarities

Using TensorFlow backend.


In [2]:
corpora_dict_path = './model/corpora_dict.dict'
tfidf_model_path = "./model/tfidf.model"
index_path = "./model/simIndex.index"

jieba.set_dictionary("dict.txt.big.txt")
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

tfidf = models.TfidfModel.load(tfidf_model_path)
dictionary = corpora.Dictionary.load(corpora_dict_path)
index = similarities.SparseMatrixSimilarity.load(index_path)

In [3]:
all_ettoday = []
for i in mycol.find({},{"title":1}):
    all_ettoday.append(i)
print(all_ettoday[:10])

[{'_id': ObjectId('5da6ec3c59aa7d15bc17ca58'), 'title': '快訊／高嘉瑜晚間好奇找阿北求證\u3000柯P終於「一句話」解答了'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca59'), 'title': '美眉的夏天／最HOT名單出爐\u3000她激戰搶下冠軍抱千元大獎'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca5a'), 'title': '搭西寧軍艦南偵\u300022名教師登太平島深化主權意識'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca5c'), 'title': '桃園區慶讚中元水燈排車繞境\u3000傳承百年特色慶典'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca61'), 'title': '鳳飛飛故事展、電影展…等紀念活動\u3000緬懷永遠的國民天后'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca5d'), 'title': '韓國瑜選總統8月3日從桃園出發\u3000要讓全國看到藍軍團結力量'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca5b'), 'title': '規劃「個人服裝包」\u3000國軍新兵服裝用APP快速搞定'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca5e'), 'title': '嘉義「火燈夜巡、大士爺祭」深獲在地認同\u3000下一步推向國際'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca62'), 'title': '台民黨「不分區名單」她第一個被點名！\u3000柯文哲傾向不選才組黨'}, {'_id': ObjectId('5da6ec3c59aa7d15bc17ca63'), 'title': '影／她吃藥睡死…起床驚見「手被蟑螂咬」模樣超慘\u3000網加碼曝：而且非常痛'}]


In [4]:
def getSims(sentence):
    sentence = re.sub("\W+","",sentence)
    texts = jieba.cut(sentence)
    new_vec = dictionary.doc2bow(texts)
    new_vec_tfidf = tfidf[new_vec]
    return index[new_vec_tfidf]

In [5]:
clean_q = lambda sentence: re.sub("\W+","",re.sub("快訊","",sentence["title"]))
all_dict = [(sentence["_id"],clean_q(sentence)) for sentence in all_ettoday]
print(all_dict[:10])

[(ObjectId('5da6ec3c59aa7d15bc17ca58'), '高嘉瑜晚間好奇找阿北求證柯P終於一句話解答了'), (ObjectId('5da6ec3c59aa7d15bc17ca59'), '美眉的夏天最HOT名單出爐她激戰搶下冠軍抱千元大獎'), (ObjectId('5da6ec3c59aa7d15bc17ca5a'), '搭西寧軍艦南偵22名教師登太平島深化主權意識'), (ObjectId('5da6ec3c59aa7d15bc17ca5c'), '桃園區慶讚中元水燈排車繞境傳承百年特色慶典'), (ObjectId('5da6ec3c59aa7d15bc17ca61'), '鳳飛飛故事展電影展等紀念活動緬懷永遠的國民天后'), (ObjectId('5da6ec3c59aa7d15bc17ca5d'), '韓國瑜選總統8月3日從桃園出發要讓全國看到藍軍團結力量'), (ObjectId('5da6ec3c59aa7d15bc17ca5b'), '規劃個人服裝包國軍新兵服裝用APP快速搞定'), (ObjectId('5da6ec3c59aa7d15bc17ca5e'), '嘉義火燈夜巡大士爺祭深獲在地認同下一步推向國際'), (ObjectId('5da6ec3c59aa7d15bc17ca62'), '台民黨不分區名單她第一個被點名柯文哲傾向不選才組黨'), (ObjectId('5da6ec3c59aa7d15bc17ca63'), '影她吃藥睡死起床驚見手被蟑螂咬模樣超慘網加碼曝而且非常痛')]


In [6]:
sims = getSims("韓國瑜宣布選總統，為人民發聲。")
print(len(sims))
print(np.argmax(sims))

Building prefix dict from D:\program\semester_final\dict.txt.big.txt ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.u202fd84447beda33a25aefd770cfa382.cache
Loading model cost 1.246 seconds.
Prefix dict has been built succesfully.


49663
28373
